In [72]:
import pyomo.environ as pe  # Pyomo environment
from idaes.core import FlowsheetBlock, StateBlock
from idaes.models.unit_models import HeatExchanger
from idaes.models.unit_models.heat_exchanger import HX0DInitializer
from idaes.models.unit_models.heat_exchanger import delta_temperature_amtd_callback
from idaes.models.properties import iapws95
import numpy as np



In [46]:
# Create an empty flowsheet and steam property parameter block.
model = pe.ConcreteModel()
model.fs = FlowsheetBlock(dynamic=False)
model.fs.properties = iapws95.Iapws95ParameterBlock()

In [47]:
# Add a hx model to the flowsheet.
model.fs.heat_exchanger = HeatExchanger(
    delta_temperature_callback=delta_temperature_amtd_callback,
    hot_side_name="shell",
    cold_side_name="tube",
    shell={"property_package": model.fs.properties},
    tube={"property_package": model.fs.properties}
)

temperatures taken from Cinti
,2020

In [48]:
# Provided data in Celsius, convert to Kelvin
T_hot_inlet_C = 662.30
T_hot_outlet_C = 192.60
T_cold_inlet_C = 20.00
T_cold_outlet_C = 496.94


T_hot_inlet = T_hot_inlet_C + 273.15  # Convert to Kelvin
T_hot_outlet = T_hot_outlet_C + 273.15  # Convert to Kelvin
T_cold_inlet = T_cold_inlet_C + 273.15  # Convert to Kelvin
T_cold_outlet = T_cold_outlet_C + 273.15  # Convert to Kelvin


Check these!

In [49]:

# Assume constant Cp for gases (ideal gas assumption)
Cp_hot = 1.039  # Specific heat of nitrogen (N2) in kJ/kg·K
Cp_cold = 1.005  # Specific heat of air at constant pressure in kJ/kg·K


Understand where we use this ref temperature

In [6]:

# Reference temperature (for simplicity, assume 0°C = 273.15 K)
T_ref = 273.15

In [50]:
# Calculate enthalpy using Cp (simplified)
def calc_enthalpy(Cp, T):
    return Cp * (T - T_ref)

In [51]:
# Calculate enthalpy for the hot and cold streams
h_hot_inlet = calc_enthalpy(Cp_hot, T_hot_inlet)  # Inlet enthalpy for the hot stream (N2/steam)
h_hot_outlet = calc_enthalpy(Cp_hot, T_hot_outlet)
h_cold_inlet = calc_enthalpy(Cp_cold, T_cold_inlet)  # Inlet enthalpy for the cold stream (air)
h_cold_outlet = calc_enthalpy(Cp_cold, T_cold_outlet)



In [52]:
# Print the enthalpy values
print(f"Hot stream inlet enthalpy (h_hot_inlet): {h_hot_inlet:.2f} kJ/kg")
print(f"Hot stream outlet enthalpy (h_hot_outlet): {h_hot_outlet:.2f} kJ/kg")
print(f"Cold stream inlet enthalpy (h_cold_inlet): {h_cold_inlet:.2f} kJ/kg")
print(f"Cold stream outlet enthalpy (h_cold_outlet): {h_cold_outlet:.2f} kJ/kg")

Hot stream inlet enthalpy (h_hot_inlet): 688.13 kJ/kg
Hot stream outlet enthalpy (h_hot_outlet): 200.11 kJ/kg
Cold stream inlet enthalpy (h_cold_inlet): 20.10 kJ/kg
Cold stream outlet enthalpy (h_cold_outlet): 499.42 kJ/kg


In [53]:
%store h_cold_inlet
%store h_cold_outlet
%store h_hot_inlet
%store h_hot_outlet


Stored 'h_cold_inlet' (float)
Stored 'h_cold_outlet' (float)
Stored 'h_hot_inlet' (float)
Stored 'h_hot_outlet' (float)


Look into molar flow rates

In [54]:
# Flow rate (Assuming constant flow rate of 100 mol/s)
flow_mol_hot = 10  # mol/s
flow_mol_cold = 10  # mol/s

In [55]:
# Convert molar flow to mass flow (assuming ideal gas behavior)
M_hot = 28.0134  # Molar mass of N2 (kg/kmol)
M_cold = 28.97  # Molar mass of air (kg/kmol)

In [56]:
mass_flow_hot = flow_mol_hot * M_hot / 1000  # Convert to kg/s
mass_flow_cold = flow_mol_cold * M_cold / 1000  # Convert to kg/s

In [57]:
# Print the mass flow rates for hot and cold streams
print(f"Mass flow rate of hot stream: {mass_flow_hot:.4f} kg/s")
print(f"Mass flow rate of cold stream: {mass_flow_cold:.4f} kg/s")

Mass flow rate of hot stream: 0.2801 kg/s
Mass flow rate of cold stream: 0.2897 kg/s


In [58]:
# Compute heat duty (Q)
Q_hot = mass_flow_hot * Cp_hot * (T_hot_inlet - T_hot_outlet)  # kJ/s = kW
Q_cold = mass_flow_cold * Cp_cold * (T_cold_outlet - T_cold_inlet)  # kJ/s = kW

In [59]:
# Print Q_hot and Q_cold values
print(f"Q_hot: {Q_hot:.2f} kW")
print(f"Q_cold: {Q_cold:.2f} kW")

Q_hot: 136.71 kW
Q_cold: 138.86 kW


In [60]:
# Ensure heat balance is satisfied
Q = min(Q_hot, Q_cold) * 1000  # Convert kW to W

In [61]:
# Print the computed Q value
print(f"Computed Q: {Q/1000:.2f} kW")

Computed Q: 136.71 kW


In [62]:
# Compute LMTD
delta_T1 = T_hot_inlet - T_cold_outlet
delta_T2 = T_hot_outlet - T_cold_inlet
LMTD = (delta_T1 - delta_T2) / np.log(delta_T1 / delta_T2)
# Print the computed LMTD
print(f"Computed LMTD: {LMTD:.2f} K")

Computed LMTD: 168.95 K


In [ ]:
# Fixed overall heat transfer coefficient from Cinti et al 2020
U = 30  # W/m²·K (30 kg/K/s³ = 30 W/m²·K)

In [64]:
# Ensure all necessary variables are fixed before initialization
model.fs.heat_exchanger.shell_inlet.flow_mol.fix(flow_mol_hot)  # Fix hot stream flow rate
model.fs.heat_exchanger.tube_inlet.flow_mol.fix(flow_mol_cold)  # Fix cold stream flow rate
model.fs.heat_exchanger.shell_inlet.enth_mol.fix(h_hot_inlet)  # Fix hot stream inlet enthalpy
model.fs.heat_exchanger.tube_inlet.enth_mol.fix(h_cold_inlet)  # Fix cold stream inlet enthalpy

model.fs.heat_exchanger.overall_heat_transfer_coefficient[0].fix(30)
model.fs.heat_exchanger.shell_inlet.pressure.fix(101325)
model.fs.heat_exchanger.tube_inlet.pressure.fix(101325)


#model.fs.heat_exchanger.area.fix()  # m²

In [39]:
# Initialize the model
initializer = HX0DInitializer()
initializer.initialize(model.fs.heat_exchanger)

2025-04-10 09:36:19 [INFO] idaes.init.fs.heat_exchanger: Initialization Completed, optimal - <undefined>


<InitializationStatus.Ok: 1>

In [65]:
from idaes.core.util.model_statistics import degrees_of_freedom
print("Degrees of Freedom:", degrees_of_freedom(model))

Degrees of Freedom: 1


In [66]:

# Ensure the solver is correctly initialized and used
solver = pe.SolverFactory("ipopt")  # Ensure SolverFactory is correctly initialized


In [67]:

results = solver.solve(model, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [75]:
model.fs.heat_exchanger.display()


Block fs.heat_exchanger

  Variables:
    overall_heat_transfer_coefficient : Overall heat transfer coefficient
        Size=1, Index=fs._time, Units=kg/K/s**3
        Key : Lower : Value : Upper : Fixed : Stale : Domain
        0.0 :     0 :    30 :  None :  True :  True : PositiveReals
    area : Heat exchange area
        Size=1, Index=None, Units=m**2
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :     0 : 99185.13829987208 :  None : False : False : PositiveReals
    delta_temperature_in : Temperature difference at the hot inlet end
        Size=1, Index=fs._time, Units=K
        Key : Lower : Value                : Upper : Fixed : Stale : Domain
        0.0 :  None : 0.002250316109490042 :  None : False : False :  Reals
    delta_temperature_out : Temperature difference at the hot outlet end
        Size=1, Index=fs._time, Units=K
        Key : Lower : Value                : Upper : Fixed : Stale : Domain
        0.0 :  None : 0.00223865993

In [68]:
A_calculated = model.fs.heat_exchanger.area

In [69]:
# Print the calculated heat exchanger area
print(f"Calculated Heat Exchanger Area: {A_calculated.value} m²")

Calculated Heat Exchanger Area: 99185.13829987208 m²


In [70]:

# Compute heat exchanger area
A = Q / (U * LMTD)

In [71]:
print(f"Computed Heat Exchanger Area: {A:.2f} m²")

Computed Heat Exchanger Area: 26.97 m²


In [31]:
%store A

Stored 'A' (float64)


UP UNTIL HERE JUST CALCULATION, FROM HERE DOWN INFO TO VISUALIZE THE FLOWSHEET WITH UI

In [32]:
# **Store these values as variables for visualization**
model.fs.heat_duty = pe.Param(initialize=Q, doc="Heat Duty (W)")  # Heat Duty
model.fs.LMTD = pe.Param(initialize=LMTD, doc="Log Mean Temperature Difference (K)")  # LMTD
model.fs.area_calculated = pe.Param(initialize=A, doc="Calculated Heat Exchanger Area (m²)")

'pyomo.core.base.param.ScalarParam'>) on block fs with a new Component
(type=<class 'pyomo.core.base.param.ScalarParam'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
'pyomo.core.base.param.ScalarParam'>) on block fs with a new Component
(type=<class 'pyomo.core.base.param.ScalarParam'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
(type=<class 'pyomo.core.base.param.ScalarParam'>) on block fs with a new
Component (type=<class 'pyomo.core.base.param.ScalarParam'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [35]:
import idaes_ui
import idaes_ui.fv
dir(idaes_ui.fv)
from idaes_ui.fv import visualize 



In [36]:

# Visualize the model
visualize(model.fs, "HeatExchangerAreaCalculation")

2025-04-10 09:35:32 [INFO] idaes.idaes_ui.fv.fsvis: Started visualization server
2025-04-10 09:35:32 [INFO] idaes.idaes_ui.fv.fsvis: Saving flowsheet to default file 'HeatExchangerAreaCalculation.json' in current directory (c:\Users\Sara\Desktop\IDAES)
2025-04-10 09:35:33 [INFO] idaes.idaes_ui.fv.fsvis: Flowsheet visualization at: http://localhost:51026/app?id=HeatExchangerAreaCalculation


VisualizeResult(store=<idaes_ui.fv.persist.FileDataStore object at 0x0000020574E557E0>, port=51026, server=<idaes_ui.fv.model_server.FlowsheetServer object at 0x0000020573E6E380>)

In [37]:
# Print results
print(f"Heat Duty (Q): {Q / 1000:.2f} kW")
print(f"LMTD: {LMTD:.2f} K")
print(f"Calculated Heat Exchanger Area: {A:.2f} m²")

Heat Duty (Q): 136.71 kW
LMTD: 168.95 K
Calculated Heat Exchanger Area: 26.97 m²
